In [ ]:
import pandas as pd

items = pd.read_pickle('~/chtl-data/processed/inventory.pkl')
in_maintenance = items[items['In Maintenance'] & ~(items['Disabled'] | items['Not Fixable'] | items['Lost In Shop'])]

In [ ]:
loans = pd.read_pickle('~/chtl-data/processed/loans.pkl')
loans_in_maintenance = loans[loans['Item ID'].isin(in_maintenance['Item ID'])]
loans_in_maintenance.groupby(by='Item ID')['Item ID'].count().sort_values(ascending=False)[:20]

In [ ]:
from datetime import datetime

loans = pd.read_pickle('~/chtl-data/processed/loans.pkl')
items = pd.read_pickle('/home/max/chtl-data/processed/inventory.pkl')
items = items.set_index("Item ID")
joined = loans.join(items, on="Item ID", lsuffix="-l")
item_type_overrides = {"Cordless Drills": "Power Drills", "Drills": "Power Drills", "Power Drill 3/8in": "Power Drills", "Flat Screwdrivers": "Screwdrivers", "Finishing Power Sanders": "Sanders"}
for k, v in item_type_overrides.items():
    joined.loc[joined["Item Type"] == k, "Item Type"] = v
    items.loc[items["Item Type"] == k, "Item Type"] = v

item_id_type_overrides = { 3894: "Automotive Tools"}
for k, v in item_id_type_overrides.items():
    joined.loc[joined["Item ID"] == k, "Item Type"] = v
    items.loc[k, "Item Type"] = v

in_maintenance = items[items['In Maintenance'] & ~(items['Disabled'] | items['Not Fixable'] | items['Lost In Shop'])]

very_overdue = loans[(loans["Due Date"] < datetime(2024, 9, 1)) & pd.isna(loans['Checked In'])]
very_overdue = very_overdue.set_index("Item ID")
# Custom functions to combine some groups
item_type_popularity = joined.groupby(["Item Type"]).size()
enabled_items = items[~(items["Disabled"] | items["Shop Use Only"] | items["Lost By Member"] | items["Lost In Shop"] | items["Not Fixable"] | items["In Maintenance"])]
items_w_overdue_info = enabled_items.join(very_overdue, rsuffix="-overdue")
item_type_count = items_w_overdue_info[items_w_overdue_info['index'].isna()].groupby(["Item Type"]).size()
with_popularity = in_maintenance.join(item_type_popularity.rename("Item Type Loan Count"), on="Item Type").join(item_type_count.rename("Item Type Available Item Count"), on="Item Type")
with_popularity["Usage Ratio"] = (with_popularity["Item Type Loan Count"] / with_popularity["Item Type Available Item Count"]).round(0)
with_popularity.sort_values(by="Usage Ratio", ascending=False)[[ "Item Type", "Name", "Disabled", "Item Type Loan Count", "Item Type Available Item Count", "Usage Ratio"]].to_csv("~/chtl-data/reports/maintenance-by-usage-ratio.csv")


In [ ]:
item_type_count[ "Power Drills"]
# items_w_overdue_info[items_w_overdue_info["Item Type"] == "Power Drills"]
# item_type_count[item_type_count["Item Type"] == "Power Drills"]
# enabled_items[enabled_items["Item ID"] == 3751]
# .join(very_overdue, on="Item ID", rsuffix="-OVERDUE")
items.iloc[1]